# HOMEWORK 1 - Chess Game
**Student:** Alessandro Mattei

**Matricola:** 295411

**Email:** alessandro.mattei1@student.univaq.it

The main components used for the implementation will be presented:
   - Class Agent
   - Class Game (ChessGame)
   - Class State (StateChessGame)
   - Heuristics (HardBoardEvaluationChessGame and SoftBoardEvaluationChessGame)
   - Observation (ObservationBoard) 
   - Search Algorithm (MinMaxAplaBetaPruning, MinMaxAlphaBetaPruningH0Cut, MinMaxAlphaBetaPruningHlCut, MinMaxAlphaBetaPruningRegressorCut)
   - Nonlinear Regressor
   - CSV generator for the nonlinear regressor

# Agent Class
Represents an agent that can act based on a given search algorithm and its current view of the world.

In [3]:
class Agent:
    """
    Represents an agent that can act based on a given search algorithm and its current view of the world.

    Attributes:
        search_algorithm: A search algorithm that the agent uses to make decisions.
        view: The agent's current view of the world.
        old_view: The agent's previous view of the world.
    """

    def __init__(self, search_algorithm, initial_state):
        """
        Initializes the Agent with a search algorithm and an initial state.

        :param search_algorithm: The search algorithm to be used by the agent.
        :param initial_state: The initial state of the world as perceived by the agent.
        """
        self.search_algorithm = search_algorithm
        self.view = initial_state
        self.old_view = None

    def do_action(self, current_state_world):
        """
        Updates the agent's view based on the current state of the world and the search algorithm.
        :param current_state_world: The current state of the world.
        :return: The updated view of the agent.
        """
        self.view = self.search_algorithm.search(current_state_world)
        self.old_view = current_state_world
        return self.view


# State
Represents a state in a chess game, including the board configuration and various heuristic evaluations.

In [5]:
import chess


class StateChessGame:
    """
    Represents a state in a chess game, including the board configuration and various heuristic evaluations.

    Attributes:
        game_board (chess.Board): The current chess board configuration.
        parent_state (StateChessGame): The parent state from which this state is derived.
        move (chess.Move): The move that led to this state.
        h (float): General heuristic value for the state.
        h0 (float): Heuristic value used for h0 cutoff.
        hl (float): Heuristic value used for hl cutoff.
        hr (float): Heuristic value used for nonlinear regressor cutoff.
    """

    def __init__(self, game_board=None, state_parent=None, move=None):
        """
        Initializes a new game state.

        :param game_board: The current chess board configuration. If None, initializes a new chess board.
        :param state_parent: The parent state from which this state is derived.
        :param move: The move that led to this state.
        """
        self.game_board = game_board  # The current chess board (chess.Board object).
        self.parent_state = state_parent  # The parent state from which this state is derived.
        self.move = move  # The move that led to this state.
        self.h = None  # General heuristic value for the state.
        self.h0 = None  # Heuristic value used for h0 cutoff.
        self.hl = None  # Heuristic value used for hl cutoff.
        self.hr = None  # Heuristic value used for nonlinear regressor cutoff.

        # If no game board is provided, initialize a new chess board.
        if self.game_board is None:
            self.game_board = chess.Board()

    def __eq__(self, other):
        """
        Checks if this state is equal to another state. States are considered equal if they have the same game
        board configuration.

        :param other: The other StateChessGame object to compare with.
        :return: True if the states are equal, False otherwise.
        """
        if not isinstance(other, StateChessGame):
            return False
        return self.game_board == other.game_board

    def __ne__(self, other):
        """
        Checks if this state is not equal to another state. It relies on the __eq__ method.

        :param other: The other StateChessGame object to compare with.
        :return: True if the states are not equal, False otherwise.
        """
        return not self.__eq__(other)

    def __hash__(self):
        """
        Generates a hash for the state. This is based on the string representation of the game board, allowing the state
        to be used in hash tables or sets.

        :return: The hash of the state.
        """
        return hash(str(self.game_board))


# Game Class
Represents a chess game, providing functionalities to manage the game state and compute possible moves.

In [6]:
import chess

from chessgame.StateChessGame import StateChessGame


class ChessGame:
    """
    Represents a chess game, providing functionalities to manage the game state and compute possible moves.

    Attributes:
        game_board (chess.Board): The current chess board configuration.
    """

    def __init__(self, game_board=None):
        """
        Initializes a new chess game.

        :param game_board: The current chess board configuration. If None, initializes a new chess board.
        """
        self.game_board = game_board  # The current chess board.

        # If no game board is provided, initialize a new chess board.
        if game_board is None:
            self.game_board = chess.Board()

    def neighbors(self, state: StateChessGame):
        """
        Generates all possible next states (neighbors) from a given state.

        :param state: The current state of the chess game from which to compute neighbors.
        :return: A list of StateChessGame objects representing possible next states.
        """
        neighbors = []

        # Iterate through all legal moves from the current state.
        for legal_move in state.game_board.legal_moves:
            # Copy the current game board and make the legal move.
            new_game_board = state.game_board.copy()
            new_game_board.push(legal_move)

            # Create a new StateChessGame object for the resulting game state.
            neighbor = StateChessGame(game_board=new_game_board, state_parent=state, move=legal_move)
            neighbors.append(neighbor)
        return neighbors

    def get_name_winner_player(self, game_board):
        """
        Determines the name of the winning player if the game is in checkmate.

        :param game_board: The chess board to check for checkmate and winner.
        :return: The name of the winning player ("White" or "Black") if there's a checkmate, otherwise None.
        """
        # Check if the current game state is a checkmate.
        if game_board.is_checkmate():
            # Get the outcome of the game.
            outcome = game_board.outcome()
            if outcome is not None:
                # Return "White" or "Black" depending on the winner.
                return "White" if outcome.winner else "Black"
        return None


# Heuristics
## Single Evaluations
### Constants

In [ ]:
import chess

PIECE_VALUE = {
    chess.PAWN: 1,
    chess.KNIGHT: 3,
    chess.BISHOP: 3,
    chess.ROOK: 5,
    chess.QUEEN: 9,
    chess.KING: 0  # Il valore del re è gestito separatamente
}

# Tabelle di posizione per il pedone
PAWN_TABLE = [
    0, 0, 0, 0, 0, 0, 0, 0,
    5, 10, 10, -20, -20, 10, 10, 5,
    5, -5, -10, 0, 0, -10, -5, 5,
    0, 0, 0, 20, 20, 0, 0, 0,
    5, 5, 10, 25, 25, 10, 5, 5,
    10, 10, 20, 30, 30, 20, 10, 10,
    50, 50, 50, 50, 50, 50, 50, 50,
    0, 0, 0, 0, 0, 0, 0, 0
]

# Tabelle di posizione per il cavallo
KNIGHT_TABLE = [
    -50, -40, -30, -30, -30, -30, -40, -50,
    -40, -20, 0, 5, 5, 0, -20, -40,
    -30, 5, 10, 15, 15, 10, 5, -30,
    -30, 0, 15, 20, 20, 15, 0, -30,
    -30, 5, 15, 20, 20, 15, 5, -30,
    -30, 0, 10, 15, 15, 10, 0, -30,
    -40, -20, 0, 0, 0, 0, -20, -40,
    -50, -40, -30, -30, -30, -30, -40, -50,
]

# Tabelle di posizione per l'alfiere
BISHOP_TABLE = [
    -20, -10, -10, -10, -10, -10, -10, -20,
    -10, 5, 0, 0, 0, 0, 5, -10,
    -10, 10, 10, 10, 10, 10, 10, -10,
    -10, 0, 10, 10, 10, 10, 0, -10,
    -10, 5, 5, 10, 10, 5, 5, -10,
    -10, 0, 5, 10, 10, 5, 0, -10,
    -10, 0, 0, 0, 0, 0, 0, -10,
    -20, -10, -10, -10, -10, -10, -10, -20
]

# Tabelle di posizione per la torre
ROOK_TABLE = [
    0, 0, 0, 5, 5, 0, 0, 0,
    -5, 0, 0, 0, 0, 0, 0, -5,
    -5, 0, 0, 0, 0, 0, 0, -5,
    -5, 0, 0, 0, 0, 0, 0, -5,
    -5, 0, 0, 0, 0, 0, 0, -5,
    -5, 0, 0, 0, 0, 0, 0, -5,
    5, 10, 10, 10, 10, 10, 10, 5,
    0, 0, 0, 0, 0, 0, 0, 0
]

# Tabelle di posizione per la regina
QUEEEN_TABLE = [
    -20, -10, -10, -5, -5, -10, -10, -20,
    -10, 0, 5, 0, 0, 0, 0, -10,
    -10, 5, 5, 5, 5, 5, 0, -10,
    0, 0, 5, 5, 5, 5, 0, -5,
    -5, 0, 5, 5, 5, 5, 0, -5,
    -10, 0, 5, 5, 5, 5, 0, -10,
    -10, 0, 0, 0, 0, 0, 0, -10,
    -20, -10, -10, -5, -5, -10, -10, -20
]

# Tabelle di posizione per il re (inizio gioco)
KING_INITGAME_TABLE = [
    20, 30, 10, 0, 0, 10, 30, 20,
    20, 20, 0, 0, 0, 0, 20, 20,
    -10, -20, -20, -20, -20, -20, -20, -10,
    -20, -30, -30, -40, -40, -30, -30, -20,
    -30, -40, -40, -50, -50, -40, -40, -30,
    -30, -40, -40, -50, -50, -40, -40, -30,
    -30, -40, -40, -50, -50, -40, -40, -30,
    -30, -40, -40, -50, -50, -40, -40, -30
]

# Tabelle di posizione per il re (fine gioco)
KING_ENDGAME_TABLE = [
    -50, -40, -30, -20, -20, -30, -40, -50,
    -30, -20, -10, 0, 0, -10, -20, -30,
    -30, -10, 20, 30, 30, 20, -10, -30,
    -30, -10, 30, 40, 40, 30, -10, -30,
    -30, -10, 30, 40, 40, 30, -10, -30,
    -30, -10, 20, 30, 30, 20, -10, -30,
    -30, -30, 0, 0, 0, 0, -30, -30,
    -50, -30, -30, -30, -30, -30, -30, -50
]

### Evaluate Board Without King

Provides heuristic evaluation of a chess board state, focusing on piece values and game conditions.

In [ ]:
class EvaluateBoardWithoutKing:
    """
    Provides heuristic evaluation of a chess board state, focusing on piece values and game conditions.

    Attributes:
        evaluate_end_game_phase (bool): Flag to indicate whether to evaluate endgame phases differently.
        normalize_result (bool): Flag to indicate whether to normalize the evaluation result.
        h_max_value (int): Maximum heuristic value for normalization.
        h_min_value (int): Minimum heuristic value for normalization.
    """

    def __init__(self, evaluate_end_game_phase=False, normalize_result=False):
        """
        Initializes the evaluator with options for endgame evaluation and result normalization.

        :param evaluate_end_game_phase: Set to True to evaluate endgame phases differently.
        :param normalize_result: Set to True to normalize the evaluation result.
        """
        self.evaluate_end_game_phase = evaluate_end_game_phase  # Flag to evaluate endgame phases.
        self.normalize_result = normalize_result  # Flag to normalize the evaluation result.
        self.h_max_value = 99  # Maximum heuristic value for normalization.
        self.h_min_value = -99  # Minimum heuristic value for normalization.

    def h(self, state: StateChessGame):
        """
        Evaluates the heuristic of a given game state.

        :param state: StateChessGame object representing the current state of the chess game.
        :return: The heuristic value of the state.
        """
        # Evaluates endgame phase or normalizes the result based on the flags set in the constructor.
        if self.evaluate_end_game_phase:
            return self.__h(state.game_board)
        elif self.normalize_result:
            raw_eval = self.__h(state.game_board)
            return self.__normalize(raw_eval)
        else:
            return self.__h(state.game_board)

    def h_piccolo(self, board):
        """
        Similar to h() but operates directly on a chess board and allows specifying normalization bounds.

        :param board: The chess board to evaluate.
        :return: The heuristic value of the board.
        """
        # Evaluates the endgame phase, normalizes the result, or provides raw evaluation.
        if self.evaluate_end_game_phase:
            return self.__h(board)
        elif self.normalize_result:
            raw_eval = self.__h(board)
            return self.__normalize(raw_eval, 10, -10)
        else:
            return self.__h(board)

    def __h(self, board):
        """
        Private method for raw heuristic evaluation of a board.

        :param board: The chess board to evaluate.
        :return: The raw heuristic value of the board.
        """
        # Special handling for endgame phase.
        if self.evaluate_end_game_phase:
            game_over_eval = None
            # Assign extreme values for checkmate situations.
            if board.is_checkmate():
                outcome = board.outcome()
                if outcome is not None:
                    game_over_eval = float("inf") if outcome.winner else float("-inf")
            # Assign zero for draw situations.
            if board.is_stalemate() or board.is_insufficient_material() or board.is_seventyfive_moves() or board.is_fivefold_repetition():
                game_over_eval = 0

            if game_over_eval is not None:
                return game_over_eval

        # If the game is over, returns high positive or negative values for checkmate, and zero for other endings.
        if board.is_game_over():
            if board.is_checkmate():
                return -99 if board.turn else 99
            else:
                return 0  # Handles stalemate and insufficient material.

        # Piece-based evaluation, optimized.
        eval = sum(PIECE_VALUE[piece] * (len(board.pieces(piece, chess.WHITE)) - len(board.pieces(piece, chess.BLACK)))
                   for piece in PIECE_VALUE)
        # Slightly favors the player whose turn it is, as they might have the initiative.
        eval += 0.1 if board.turn else -0.1

        return eval

    def __normalize(self, value, maxv=100, minv=-100):
        """
        Normalizes the evaluation value within a specified range.

        :param value: The value to be normalized.
        :param maxv: The maximum value for normalization. Defaults to 100.
        :param minv: The minimum value for normalization. Defaults to -100.
        :return: The normalized value.
        """
        # Normalizes the value within the range from minv to maxv.
        if value >= 0:
            # Normalizes positive values.
            normalized = (value / self.h_max_value) * 100
        else:
            # Normalizes negative values.
            normalized = (value / abs(self.h_min_value)) * 100

        # Limits the normalized value between minv and maxv.
        normalized = max(min(normalized, maxv), minv)
        return normalized


## Evaluate Central Control Score

Provides heuristic evaluation of a chess board state with a focus on the control of central squares.

In [ ]:
class EvaluateCentralControlScore:
    """
    Provides heuristic evaluation of a chess board state with a focus on the control of central squares.

    Attributes:
        evaluate_end_game_phase (bool): Flag to indicate whether to evaluate endgame phases differently.
        normalize_result (bool): Flag to indicate whether to normalize the evaluation result.
        h_max_value (float): Maximum heuristic value for normalization.
        h_min_value (float): Minimum heuristic value for normalization.
    """

    def __init__(self, evaluate_end_game_phase=False, normalize_result=False):
        """
        Initializes the evaluator with options for endgame evaluation and result normalization.

        :param evaluate_end_game_phase: Set to True to evaluate endgame phases differently.
        :param normalize_result: Set to True to normalize the evaluation result.
        """
        self.evaluate_end_game_phase = evaluate_end_game_phase  # Flag to evaluate endgame phases.
        self.normalize_result = normalize_result  # Flag to normalize the evaluation result.
        self.h_max_value = 1.2  # Maximum heuristic value for normalization.
        self.h_min_value = -1.2  # Minimum heuristic value for normalization.

    def h(self, state: StateChessGame):
        """
        Evaluates the heuristic of a given game state.

        :param state: StateChessGame object representing the current state of the chess game.
        :return: The heuristic value of the state.
        """
        # Evaluates endgame phase or normalizes the result based on the constructor's flags.
        if self.evaluate_end_game_phase:
            return self.__h(state.game_board)
        elif self.normalize_result:
            raw_eval = self.__h(state.game_board)
            return self.__normalize(raw_eval)
        else:
            return self.__h(state.game_board)

    def h_piccolo(self, board):
        """
        Similar to h() but operates directly on a chess board and allows specifying normalization bounds.

        :param board: The chess board to evaluate.
        :return: The heuristic value of the board.
        """
        # Evaluates the endgame phase, normalizes the result, or provides raw evaluation.
        if self.evaluate_end_game_phase:
            return self.__h(board)
        elif self.normalize_result:
            raw_eval = self.__h(board)
            return self.__normalize(raw_eval, 10, -10)
        else:
            return self.__h(board)

    def __h(self, board):
        """
        Private method for raw heuristic evaluation of a board.

        :param board: The chess board to evaluate.
        :return: The raw heuristic value of the board.
        """
        # Special handling for endgame phase.
        if self.evaluate_end_game_phase:
            game_over_eval = None
            # Assign extreme values for checkmate situations.
            if board.is_checkmate():
                outcome = board.outcome()
                if outcome is not None:
                    game_over_eval = float("inf") if outcome.winner else float("-inf")
            # Assign zero for draw situations.
            if board.is_stalemate() or board.is_insufficient_material() or board.is_seventyfive_moves() or board.is_fivefold_repetition():
                game_over_eval = 0

            if game_over_eval is not None:
                return game_over_eval

        # Assign points for control of each central square.
        center_squares = [chess.D4, chess.E4, chess.D5, chess.E5]
        score = 0
        central_square_control = 0.3  # Value to calibrate based on your specific heuristic.

        # Check if central squares are controlled by White or Black.
        for square in center_squares:
            if board.is_attacked_by(chess.WHITE, square):
                score += central_square_control
            if board.is_attacked_by(chess.BLACK, square):
                score -= central_square_control

        # Adjust the score for the current player.
        return score if board.turn == chess.WHITE else -score

    def __normalize(self, value, maxv=100, minv=-100):
        """
        Normalizes the evaluation value within a specified range.

        :param value: The value to be normalized.
        :param maxv: The maximum value for normalization. Defaults to 100.
        :param minv: The minimum value for normalization. Defaults to -100.
        :return: The normalized value.
        """
        # Normalizes the value within the range from minv to maxv.
        if value >= 0:
            # Normalizes positive values.
            normalized = (value / self.h_max_value) * 100
        else:
            # Normalizes negative values.
            normalized = (value / abs(self.h_min_value)) * 100

        # Limits the normalized value between minv and maxv.
        normalized = max(min(normalized, maxv), minv)
        return normalized

## 

## SoftBoardEvaluationChessGame
Provides an evaluation of a chess board based on various criteria, helping to determine the quality of a board state for use in search algorithms.
This heuristic is the simplest implemented.
Combine various heuristics by summing the value of them.

Combined heuristics:
   - evaluate_board: Evaluates the overall quality of the board.
   - material_evaluation: Evaluates the board based on the material present.
   - piece_square_evaluation: Evaluates the board based on piece positions.
   - mobility_evaluation: Evaluates the board based on piece mobility.
   - king_safety_evaluation: Evaluates the board based on king safety.
   - center_control_evaluation: Evaluates board control of center squares.

In [7]:
import chess

PIECE_VALUES = {
    chess.PAWN: 1,
    chess.KNIGHT: 3,
    chess.BISHOP: 3,
    chess.ROOK: 5,
    chess.QUEEN: 9,
    chess.KING: 0  # Il re ha un valore speciale
}

# Tabelle di pezzi
PAWN_TABLE = [
    [0, 0, 0, 0, 0, 0, 0, 0],
    [5, 10, 10, -20, -20, 10, 10, 5],
    [5, -5, -10, 0, 0, -10, -5, 5],
    [0, 0, 0, 20, 20, 0, 0, 0],
    [5, 5, 10, 25, 25, 10, 5, 5],
    [10, 10, 20, 30, 30, 20, 10, 10],
    [50, 50, 50, 50, 50, 50, 50, 50],
    [0, 0, 0, 0, 0, 0, 0, 0]
]

KNIGHT_TABLE = [
    [-50, -40, -30, -30, -30, -30, -40, -50],
    [-40, -20, 0, 5, 5, 0, -20, -40],
    [-30, 5, 10, 15, 15, 10, 5, -30],
    [-30, 0, 15, 20, 20, 15, 0, -30],
    [-30, 5, 15, 20, 20, 15, 5, -30],
    [-30, 0, 10, 15, 15, 10, 0, -30],
    [-40, -20, 0, 0, 0, 0, -20, -40],
    [-50, -40, -30, -30, -30, -30, -40, -50]
]

BISHOP_TABLE = [
    [-20, -10, -10, -10, -10, -10, -10, -20],
    [-10, 5, 0, 0, 0, 0, 5, -10],
    [-10, 10, 10, 10, 10, 10, 10, -10],
    [-10, 0, 10, 10, 10, 10, 0, -10],
    [-10, 5, 5, 10, 10, 5, 5, -10],
    [-10, 0, 5, 10, 10, 5, 0, -10],
    [-10, 0, 0, 0, 0, 0, 0, -10],
    [-20, -10, -10, -10, -10, -10, -10, -20]
]

ROOK_TABLE = [
    [0, 0, 0, 5, 5, 0, 0, 0],
    [-5, 0, 0, 0, 0, 0, 0, -5],
    [-5, 0, 0, 0, 0, 0, 0, -5],
    [-5, 0, 0, 0, 0, 0, 0, -5],
    [-5, 0, 0, 0, 0, 0, 0, -5],
    [-5, 0, 0, 0, 0, 0, 0, -5],
    [5, 10, 10, 10, 10, 10, 10, 5],
    [0, 0, 0, 0, 0, 0, 0, 0]
]

QUEEN_TABLE = [
    [-20, -10, -10, -5, -5, -10, -10, -20],
    [-10, 0, 0, 0, 0, 0, 0, -10],
    [-10, 0, 5, 5, 5, 5, 0, -10],
    [-5, 0, 5, 5, 5, 5, 0, -5],
    [0, 0, 5, 5, 5, 5, 0, -5],
    [-10, 5, 5, 5, 5, 5, 0, -10],
    [-10, 0, 5, 0, 0, 0, 0, -10],
    [-20, -10, -10, -5, -5, -10, -10, -20]
]

KING_TABLE = [
    [20, 30, 10, 0, 0, 10, 30, 20],
    [20, 20, 0, 0, 0, 0, 20, 20],
    [-10, -20, -20, -20, -20, -20, -20, -10],
    [-20, -30, -30, -40, -40, -30, -30, -20],
    [-30, -40, -40, -50, -50, -40, -40, -30],
    [-30, -40, -40, -50, -50, -40, -40, -30],
    [-30, -40, -40, -50, -50, -40, -40, -30],
    [-30, -40, -40, -50, -50, -40, -40, -30]
]

# Pesi, Penalità e bonus
OPEN_FILE_PENALTY = -25
ADJACENT_PAWN_BONUS = 10
KING_SAFETY_WEIGHT = 0.5
MOBILITY_WEIGHT = 0.1
CENTER_CONTROL_WEIGHT = 0.5
CENTER_SQUARES = [chess.D4, chess.D5, chess.E4, chess.E5]


class SoftBoardEvaluationChessGame:
    """
    Provides an evaluation of a chess board based on various criteria, helping
    to determine the quality of a board state for use in search algorithms.

    Methods:
        evaluate_board: Evaluates the overall quality of the board.
        material_evaluation: Evaluates the board based on the material present.
        piece_square_evaluation: Evaluates the board based on piece positions.
        mobility_evaluation: Evaluates the board based on piece mobility.
        king_safety_evaluation: Evaluates the board based on king safety.
        center_control_evaluation: Evaluates board control of center squares.
    """

    def h(self, state: StateChessGame):
        """
        Evaluates the overall quality of the board based on various criteria.
        :param state:The current state of the chess game.
        :return: The evaluation score of the board.
        """
        board = state.game_representation.game_board
        h1 = state.game_over_eval()
        if h1 is not None:
            return h1
        else:
            total_evaluation = (
                    self.piece_material_evaluation(board) +
                    self.piece_position_evaluation(board) +
                    self.mobility_evaluation(board) +
                    self.king_safety_evaluation(board) +
                    self.center_control_evaluation(board)
            )
            return total_evaluation

    def piece_material_evaluation(self, board):
        """
        Evaluates the board based on the material (pieces) present.

        This function calculates a numerical evaluation score for a given chess board based on the material (pieces)
        present on the board. It assigns scores to pieces based on their type and color, and the final score reflects
        the material advantage or disadvantage of one side over the other.

        :param board: The current chess board (chess.Board object). :return: The evaluation score based on material.
        A positive score indicates an advantage for white, while a negative score indicates an advantage for black.
        """
        evaluation = 0.0
        # Iterate through all squares on the chess board and evaluate the material present on each square.
        for square, piece in board.piece_map().items():
            # Get the value of the chess piece based on its type (pawn, knight, bishop, rook, queen, king).
            piece_value = PIECE_VALUES[piece.piece_type]

            # Check if the piece is white (color is chess.WHITE) or black (color is chess.BLACK) and adjust the
            # evaluation score accordingly.
            if piece.color == chess.WHITE:
                evaluation += piece_value  # Add the piece value for white.
            else:
                evaluation -= piece_value  # Subtract the piece value for black.
        return evaluation

    def piece_position_evaluation(self, board):
        """
        Evaluates the board based on the positions of the pieces.

        This function calculates a numerical evaluation score for a given chess board based on the positions of the
        pieces. It assigns scores to pieces based on their positions using predefined tables.

        :param board: The current chess board (chess.Board object). :return: The evaluation score based on piece
        positions. A positive score indicates an advantage for white based on piece positions, while a negative score
        indicates an advantage for black.
        """
        evaluation = 0.0

        for square, piece in board.piece_map().items():
            # Determine which piece type (pawn, knight, bishop, rook, queen, king) is on the current square.
            if piece.piece_type == chess.PAWN:
                table = PAWN_TABLE
            elif piece.piece_type == chess.KNIGHT:
                table = KNIGHT_TABLE
            elif piece.piece_type == chess.BISHOP:
                table = BISHOP_TABLE
            elif piece.piece_type == chess.ROOK:
                table = ROOK_TABLE
            elif piece.piece_type == chess.QUEEN:
                table = QUEEN_TABLE
            elif piece.piece_type == chess.KING:
                table = KING_TABLE

            # Calculate the row and column of the square.
            row = square // 8
            col = square % 8

            # Check if the piece is white (color is chess.WHITE) or black (color is chess.BLACK) and adjust the
            # evaluation score accordingly.
            if piece.color == chess.WHITE:
                evaluation += table[row][col]  # Add the piece value for white.
            else:
                # Tables are made for white, so let's reverse for black
                evaluation -= table[7 - row][col]  # Subtract the piece value for black.

        return evaluation

    def mobility_evaluation(self, board):
        """
        Evaluates the board based on the mobility (legal moves) of the pieces. This function calculates a numerical
        evaluation score for a given chess board based on the mobility of the pieces.
        Mobility refers to the number of legal moves that can be made by each side (white and black) on the board.

        :param board: The current chess board (chess.Board object).
        :return: The evaluation score based on mobility. A positive score indicates an advantage for the side with more
                 mobility, while a negative score indicates an advantage for the side with less mobility.
        """
        evaluation = 0.0

        # Calculate mobility for white and black
        white_mobility = len(list(board.legal_moves))
        board.push(chess.Move.null())  # Perform a null move to change the turn
        black_mobility = len(list(board.legal_moves))
        board.pop()  # Go back to the original shift
        # Evaluate mobility based on weights
        evaluation += MOBILITY_WEIGHT * (white_mobility - black_mobility)

        return evaluation

    def king_safety_evaluation(self, board):
        """
        Evaluates the safety of kings on the board.

        This function calculates a numerical evaluation score for a given chess board based on the safety of both kings.
        It considers factors such as pawn protection and open files near the kings.

        :param board: The current chess board (chess.Board object).
        :return: The evaluation score based on king safety. A positive score indicates a safer position for the white
                 king, while a negative score indicates a safer position for the black king.
        """
        evaluation = 0.0

        # Find the positions of the white and black kings on the board
        white_king_square = list(board.pieces(chess.KING, chess.WHITE))[0]
        black_king_square = list(board.pieces(chess.KING, chess.BLACK))[0]

        # Evaluate the safety of the white king
        if board.attacks(white_king_square) & board.pieces(chess.PAWN, chess.BLACK):
            # If black pawns can attack the white king, penalize the evaluation (open file penalty).
            evaluation += OPEN_FILE_PENALTY
        # Check adjacent squares to the white king for friendly pawns and provide a bonus for pawn protection.
        for square in chess.SQUARES:
            if abs(square - white_king_square) in [1, 7, 8, 9] and board.piece_at(
                    square) == chess.PAWN and board.color_at(square) == chess.WHITE:
                evaluation += ADJACENT_PAWN_BONUS

        # Evaluate the safety of the black king
        if board.attacks(black_king_square) & board.pieces(chess.PAWN, chess.WHITE):
            # If white pawns can attack the black king, penalize the evaluation (open file penalty).
            evaluation -= OPEN_FILE_PENALTY
        # Check adjacent squares to the black king for friendly pawns and provide a bonus for pawn protection.
        for square in chess.SQUARES:
            if abs(square - black_king_square) in [1, 7, 8, 9] and board.piece_at(
                    square) == chess.PAWN and board.color_at(square) == chess.BLACK:
                evaluation -= ADJACENT_PAWN_BONUS

        return evaluation * KING_SAFETY_WEIGHT

    def center_control_evaluation(self, board):
        """
        Evaluates control of the center of the chessboard.

        This function calculates a numerical evaluation score for a given chess board based on control of the central squares
        of the board. It assigns scores to pieces occupying central squares and gives additional scores for pieces
        controlling central squares.

        :param board: The current chess board (chess.Board object).
        :return: The evaluation score based on control of the center. A positive score indicates better control of the center
                 by white, while a negative score indicates better control by black.
        """

        evaluation = 0.0

        for square in CENTER_SQUARES:
            # If a central square is occupied by a piece, assign a score based on the piece color.
            piece = board.piece_at(square)
            if piece:
                if piece.color == chess.WHITE:
                    evaluation += 1
                else:
                    evaluation -= 1

            # Assign additional scores based on the number of attackers to central squares by each side.
            attackers = board.attackers(chess.WHITE, square)
            evaluation += len(attackers)
            attackers = board.attackers(chess.BLACK, square)
            evaluation -= len(attackers)

        return evaluation * CENTER_CONTROL_WEIGHT


## HardBoardEvaluationChessGame
Provides an evaluation of a chess board based on various criteria, helping to determine the quality of a board state for use in search algorithms.
This heuristic is the most complex implemented.
Combine various heuristics by summing the value of them.

Combined heuristics:
   - king_safety: Evaluates the board based on king safety.
   - all_piece_values_and_piece_square_tables: Evaluates the overall quality of the board.
   - center_control: Evaluate control of the central squares on the chessboard.
   - mobility: Evaluate the mobility of pieces on the chessboard.
   - attack_value: Evaluate the value of piece attacks on the chessboard.
   - rooks_on_open_files: Evaluate the presence of rooks on open files in the chessboard.
   - check_forks: Evaluate the presence of fork opportunities in the chess position.
   - check_pins: Evaluate the presence of pinned pieces in the chess position.

In [8]:
# Dictionary defining the intrinsic values for each chess piece.
piece_values = {
    "p": 100,  # Value of a Pawn
    "n": 320,  # Value of a Knight
    "b": 330,  # Value of a Bishop
    "r": 500,  # Value of a Rook
    "q": 900,  # Value of a Queen
    "k": 20000,  # Value of a King (set very high to represent its critical importance)
}

# Piece-square table for the white pawn, defining values based on pawn's position on the board.
pawn_white_table = [
    0, 0, 0, 0, 0, 0, 0, 0,
    5, 10, 10, -20, -20, 10, 10, 5,
    5, -5, -10, 0, 0, -10, -5, 5,
    0, 0, 0, 20, 20, 0, 0, 0,
    5, 5, 10, 25, 25, 10, 5, 5,
    10, 10, 20, 30, 30, 20, 10, 10,
    50, 50, 50, 50, 50, 50, 50, 50,
    0, 0, 0, 0, 0, 0, 0, 0
]

# The black pawn's piece-square table is just a reversed version of the white pawn's table.
pawn_black_table = list(reversed(pawn_white_table))

# Piece-square table for the white knight.
knight_white_table = [
    -50, -40, -30, -30, -30, -30, -40, -50,
    -40, -20, 0, 5, 5, 0, -20, -40,
    -30, 5, 10, 15, 15, 10, 5, -30,
    -30, 0, 15, 20, 20, 15, 0, -30,
    -30, 5, 15, 20, 20, 15, 5, -30,
    -30, 0, 10, 15, 15, 10, 0, -30,
    -40, -20, 0, 0, 0, 0, -20, -40,
    -50, -40, -30, -30, -30, -30, -40, -50
]

# The black knight's table is a reversed version of the white knight's table.
knight_black_table = list(reversed(knight_white_table))

# Piece-square table for the white bishop.
bishop_white_table = [
    -20, -10, -10, -10, -10, -10, -10, -20,
    -10, 5, 0, 0, 0, 0, 5, -10,
    -10, 10, 10, 10, 10, 10, 10, -10,
    -10, 0, 10, 10, 10, 10, 0, -10,
    -10, 5, 5, 10, 10, 5, 5, -10,
    -10, 0, 5, 10, 10, 5, 0, -10,
    -10, 0, 0, 0, 0, 0, 0, -10,
    -20, -10, -10, -10, -10, -10, -10, -20
]

# The black bishop's table is a reversed version of the white bishop's table.
bishop_black_table = list(reversed(bishop_white_table))

# Piece-square table for the white rook.
rook_white_table = [
    0, 0, 0, 5, 5, 0, 0, 0,
    -5, 0, 0, 0, 0, 0, 0, -5,
    -5, 0, 0, 0, 0, 0, 0, -5,
    -5, 0, 0, 0, 0, 0, 0, -5,
    -5, 0, 0, 0, 0, 0, 0, -5,
    -5, 0, 0, 0, 0, 0, 0, -5,
    5, 10, 10, 10, 10, 10, 10, 5,
    0, 0, 0, 0, 0, 0, 0, 0
]

# The black rook's table is a reversed version of the white rook's table.
rook_black_table = list(reversed(rook_white_table))

# Piece-square table for the white queen.
queen_white_table = [
    -20, -10, -10, -5, -5, -10, -10, -20,
    -10, 0, 0, 0, 0, 0, 0, -10,
    -10, 5, 5, 5, 5, 5, 0, -10,
    0, 0, 5, 5, 5, 5, 0, -5,
    -5, 0, 5, 5, 5, 5, 0, -5,
    -10, 0, 5, 5, 5, 5, 0, -10,
    -10, 0, 0, 0, 0, 0, 0, -10,
    -20, -10, -10, -5, -5, -10, -10, -20
]

# The black queen's table is a reversed version of the white queen's table.
queen_black_table = list(reversed(queen_white_table))

# Piece-square table for the white king during the middle game.
king_white_table = [
    20, 30, 10, 0, 0, 10, 30, 20,
    20, 20, 0, 0, 0, 0, 20, 20,
    -10, -20, -20, -20, -20, -20, -20, -10,
    -20, -30, -30, -40, -40, -30, -30, -20,
    -30, -40, -40, -50, -50, -40, -40, -30,
    -30, -40, -40, -50, -50, -40, -40, -30,
    -30, -40, -40, -50, -50, -40, -40, -30,
    -30, -40, -40, -50, -50, -40, -40, -30
]

# The black king's table is a reversed version of the white king's table.
king_black_table = list(reversed(king_white_table))

# Piece-square table for the white king during the endgame.
king_white_table_endgame = [
    -50, -30, -30, -30, -30, -30, -30, -50,
    -30, -30, 0, 0, 0, 0, -30, -30,
    -30, -10, 20, 30, 30, 20, -10, -30,
    -30, -10, 30, 40, 40, 30, -10, -30,
    -30, -10, 30, 40, 40, 30, -10, -30,
    -30, -10, 20, 30, 30, 20, -10, -30,
    -30, -20, -10, 0, 0, -10, -20, -30,
    -50, -40, -30, -20, -20, -30, -40, -50
]

# The black king's endgame table is a reversed version of the white king's endgame table.
king_black_table_endgame = list(reversed(king_white_table_endgame))

# A comprehensive dictionary containing piece-square tables for each piece and color.
# The tables indicate the value of placing a piece on a specific square.
piece_square_tables = {
    "p": pawn_black_table,  # Black pawn
    "n": knight_black_table,  # Black knight
    "b": bishop_black_table,  # Black bishop
    "r": rook_black_table,  # Black rook
    "q": queen_black_table,  # Black queen
    "k": {"early": king_black_table, "end": king_black_table_endgame},  # Black king (both middle game and endgame)

    "P": pawn_white_table,  # White pawn
    "N": knight_white_table,  # White knight
    "B": bishop_white_table,  # White bishop
    "R": rook_white_table,  # White rook
    "Q": queen_white_table,  # White queen
    "K": {"early": king_white_table, "end": king_white_table_endgame},  # White king (both middle game and endgame)
}


class HardBoardEvaluationChessGame:
    """
    Provides an evaluation of a chess board based on various criteria, helping
    to determine the quality of a board state for use in search algorithms.

    Methods:
        king_safety: Evaluates the board based on king safety.
        all_piece_values_and_piece_square_tables: Evaluates the overall quality of the board.
        center_control: Evaluate control of the central squares on the chessboard.
        mobility: Evaluate the mobility of pieces on the chessboard.
        attack_value: Evaluate the value of piece attacks on the chessboard.
        rooks_on_open_files: Evaluate the presence of rooks on open files in the chessboard.
        check_forks: Evaluate the presence of fork opportunities in the chess position.
        check_pins: Evaluate the presence of pinned pieces in the chess position.
    """

    def h(self, state: StateChessGame):
        """
        Evaluates the overall quality of the board based on various criteria.
        :param state:The current state of the chess game.
        :return: The evaluation score of the board.
        """
        h1 = state.game_over_eval()
        if h1 is not None:
            return h1
        else:
            return (
                    self.king_safety(state) +
                    self.all_piece_values_and_piece_square_tables(state) +
                    self.center_control(state) +
                    self.mobility(state) +
                    self.attack_value(state) +
                    self.rooks_on_open_files(state) +
                    self.check_forks(state) +
                    self.check_pins(state)
            )

    def king_safety(self, state: StateChessGame):
        """
        Evaluates king safety for both sides.

        This function calculates the king safety evaluation based on the positions of kings and rooks. It penalizes the
        side if its king is on an open file with an opposing rook.

        :param state: The current state of the chess game (StateChessGame object).
        :return: The evaluation score for king safety. Positive score indicates safer kings for white, and negative score
                 indicates safer kings for black.
        """
        board = state.game_representation.game_board
        value = 0
        king_positions = {'K': board.king(chess.WHITE), 'k': board.king(chess.BLACK)}
        rook_positions = {'R': list(board.pieces(chess.ROOK, chess.WHITE)),
                          'r': list(board.pieces(chess.ROOK, chess.BLACK))}
        for rook_pos in rook_positions['r']:
            if king_positions['K'] and (
                    rook_pos // 8 == king_positions['K'] // 8 or rook_pos % 8 == king_positions['K'] % 8):
                value -= 50

        for rook_pos in rook_positions['R']:
            if king_positions['k'] and (
                    rook_pos // 8 == king_positions['k'] // 8 or rook_pos % 8 == king_positions['k'] % 8):
                value += 50

        return value * 0.8

    def all_piece_values_and_piece_square_tables(self, state: StateChessGame):
        """
        Calculate the combined value of all pieces on the chessboard.

        This function computes the total value of all pieces on the chessboard, considering their intrinsic values and
        positional advantages or disadvantages based on piece-square tables. It accounts for both the middle game and endgame
        scenarios.

        :param state: The current state of the chess game (StateChessGame object).
        :return: The combined evaluation score for all pieces on the board. A positive score indicates an advantage for white,
                 and a negative score indicates an advantage for black.
        """
        total = 0
        endgame = state.game_representation.is_in_endgame_phase()
        # Iterate through all squares on the chessboard and evaluate the value of pieces on each square.
        for square, piece in state.game_representation.game_board.piece_map().items():
            piece_str = str(piece)
            piece_type = piece_str.lower()
            piece_value = 0
            if piece.piece_type == chess.KING:
                if not endgame:
                    piece_value = (
                            piece_values[piece_type]
                            + piece_square_tables[piece_str]["early"][square]
                    )
                else:
                    piece_value = (
                            piece_values[piece_type]
                            + piece_square_tables[piece_str]["end"][square]
                    )
            else:
                piece_value = (
                        piece_values[piece_type] + piece_square_tables[piece_str][square]
                )
            # Add or subtract the piece value based on its color (white or black).
            total += piece_value if piece.color == chess.WHITE else -piece_value
        return total

    def center_control(self, state: StateChessGame):
        """
        Evaluate control of the central squares on the chessboard.

        This function calculates an evaluation score based on the control of central squares on the chessboard. It awards
        points for pieces occupying or influencing central squares, with a bonus for pieces controlled by the player (white)
        and a penalty for pieces controlled by the opponent (black).

        :param state: The current state of the chess game (StateChessGame object).
        :return: The evaluation score for central control. A positive score indicates an advantage for white, and a negative
                 score indicates an advantage for black.
        """
        board = state.game_representation.game_board  # Access the chess board from the game state.
        center_squares = [chess.D3, chess.E3, chess.D4, chess.E4]
        value = 0
        for square in center_squares:
            piece = board.piece_at(square)
            if piece:
                value += 10 if piece.symbol().isupper() else -10
        return value * 0.4

    def mobility(self, state: StateChessGame):
        """
        Evaluate the mobility of pieces on the chessboard.

        This function calculates an evaluation score based on the mobility of pieces on the chessboard. It assesses
        the number of legal moves available to each piece, considering their types (pawn, knight, bishop, rook,
        queen, king) and assigns scores accordingly. Mobility is a key factor in evaluating a position's strength.

        :param state: The current state of the chess game (StateChessGame object). :return: The evaluation score for
                piece mobility. A positive score indicates an advantage for white, and a negative score indicates an
                advantage for black.
        """
        # Define piece mobility values, specifying the importance of mobility for each piece type.
        piece_mobility_values = {
            chess.PAWN: 1,
            chess.KNIGHT: 3,
            chess.BISHOP: 3,
            chess.ROOK: 2,
            chess.QUEEN: 1,
            chess.KING: 1
        }

        board = state.game_representation.game_board
        mobility_value = 0

        # Iterate through all squares on the chessboard.
        for square in chess.SQUARES:
            piece = board.piece_at(square)
            if piece:
                # Calculate the legal moves for the current piece.
                legal_moves = board.attacks(square)
                num_moves = len(legal_moves)
                # Calculate a score based on the number of legal moves and the piece's type.
                score = num_moves * piece_mobility_values.get(piece.piece_type, 0)
                # Add or subtract the score based on the piece's color (white or black).
                mobility_value += score if piece.color == board.turn else -score

        return mobility_value * 0.6

    def attack_value(self, state: StateChessGame):
        """
        Evaluate the value of piece attacks on the chessboard.

        This function calculates an evaluation score based on the value of piece attacks on the chessboard. It assesses the
        value of pieces that are attacking or defending squares and considers whether a check is present in the position.

        :param state: The current state of the chess game (StateChessGame object).
        :return: The evaluation score for attack value. A positive score indicates an advantage for white, and a negative
                 score indicates an advantage for black.
        """
        board = state.game_representation.game_board
        value = 0
        # Iterate through all squares on the chessboard.
        for square in chess.SQUARES:
            piece = board.piece_at(square)
            if piece:
                # Determine the value of the piece on the current square.
                attacked_value = piece_values[piece.symbol().lower()]
                # Find attackers of the square from the opponent's side.
                attackers_of_square = board.attackers(not board.turn, square)
                if piece.color == board.turn:
                    # Subtract the value of attackers if the piece belongs to the player (white).
                    value -= len(attackers_of_square) * attacked_value
                else:
                    # Add the value of attackers if the piece belongs to the opponent (black).
                    value += len(attackers_of_square) * attacked_value
        # Add a bonus if the position is in check.
        if board.is_check():
            value += 20
        return value * 0.8

    def rooks_on_open_files(self, state: StateChessGame):
        """
        Evaluate the presence of rooks on open files in the chessboard.

        This function calculates an evaluation score based on the presence of rooks on open files in the chessboard. It assesses
        each column (file) to check if it is open (no pawns blocking) and whether there is at least one rook present. A bonus
        or penalty is assigned depending on the color of the rook (white or black) and whether the file is open or not.

        :param state: The current state of the chess game (StateChessGame object).
        :return: The evaluation score for rooks on open files. A positive score indicates an advantage for white, and a negative
                 score indicates an advantage for black.
        """
        board = state.game_representation.game_board
        value = 0
        # Iterate through each column (file) on the chessboard.
        for x in range(8):
            has_rook = False
            open_file = True
            rook_color = None
            # Iterate through each row (rank) in the current column.
            for y in range(8):
                piece = board.piece_at(8 * y + x)
                if piece and piece.symbol() in ['P', 'p']:
                    # If a pawn is found, the file is not open.
                    open_file = False
                if piece and piece.symbol() in ['R', 'r']:
                    # If a rook is found, mark that a rook is present on this file.
                    has_rook = True
                    rook_color = piece.color
            # Check if the file is open and there is at least one rook present.
            if open_file and has_rook:
                # Assign a bonus or penalty based on the color of the rook.
                value += 25 if rook_color == chess.WHITE else -25
        return value * 0.4

    def check_forks(self, state: StateChessGame):
        """
        Evaluate the presence of fork opportunities in the chess position.

        This function calculates an evaluation score based on the presence of fork opportunities in the chess
        position. It considers legal moves for the current player and checks whether each move results in multiple
        attackers on a single square, potentially creating a fork. A bonus is assigned for each detected fork
        opportunity.

        :param state: The current state of the chess game (StateChessGame object).
        :return: The evaluation score for fork opportunities. A positive score indicates an advantage for white,
                and a negative score indicates an advantage for black.
        """
        board = state.game_representation.game_board
        forks_value = 0
        # Iterate through all legal moves for the current player.
        for move in board.legal_moves:
            # Make the move on the board temporarily.
            board.push(move)
            # Find attackers after the move to the destination square.
            attacks_after_move = board.attackers(board.turn, move.to_square)
            # Check if multiple attackers are present on the same square, indicating a fork opportunity.
            if len(attacks_after_move) > 1:
                forks_value += 10
            # Undo the move to explore other moves.
            board.pop()
        return forks_value * 0.6

    def check_pins(self, state: StateChessGame):
        """
        Evaluate the presence of pinned pieces in the chess position.

        This function calculates an evaluation score based on the presence of pinned pieces in the chess position. It
        checks each square on the chessboard to identify pieces that belong to the current player (not opponent) and
        determines if any of those pieces are pinned by an opponent's piece. A penalty is assigned for each detected
        pinned piece.

        :param state: The current state of the chess game (StateChessGame object).
        :return: The evaluation score for pinned pieces. A positive score indicates an advantage for white,
                and a negative score indicates an advantage for black.
        """
        board = state.game_representation.game_board
        pins_value = 0
        opponent_color = not board.turn
        # Iterate through all squares on the chessboard.
        for square in chess.SQUARES:
            piece = board.piece_at(square)
            # Check if there is a piece on the square, and if it belongs to the current player.
            if piece and piece.color == board.turn:
                # Find attackers of the square by the opponent.
                attackers = board.attackers(opponent_color, square)
                for attacker_square in attackers:
                    # Check if the piece on the square is pinned.
                    attacker_piece = board.piece_at(attacker_square)
                    if board.is_pinned(board.turn, attacker_square):
                        # If pinned, assign a penalty to the evaluation score.
                        pins_value -= 10

        return pins_value * 0.4


# Search Algorithm
## MinMax

In [9]:
class MinMax:
    """
    Implementation of the Minimax algorithm for game state evaluation and decision-making.

    Attributes:
        game: An instance of a game object which provides interface methods for the game state and its neighbors.
        heuristic: An instance of a heuristic object used to evaluate game states.
        max_depth: The maximum depth for the minimax search. Default is 1.
        eval_count: Count of the evaluations performed during the search.
    """

    def __init__(self, game, heuristic, max_depth=1):
        """
        Initializes an instance of the MinMax class.
        :param game: The game for which the search is performed.
        :param heuristic: The heuristic used to evaluate the game states.
        :param max_depth: The maximum depth of the search. Default is 1.
        """
        self.game = game
        self.heuristic = heuristic
        self.max_depth = max_depth
        self.eval_count = 0

    @staticmethod
    def pick(states, parent_turn):
        """
        Picks the best state based on the heuristic values.

        This static method selects the best game state from a list of states based on their heuristic values.
        The selection is determined by whether it's the maximizing player's turn or the minimizing player's turn.

        :param states: List of game states to pick from.
        :param parent_turn: Indicates whose turn it is: True for the player trying to maximize and False for
                            the player trying to minimize.
        :return: The best state based on the heuristic value.
        """
        if parent_turn:
            return max(states, key=lambda state: state.h)  # Select the state with the highest heuristic value.
        else:
            return min(states, key=lambda state: state.h)  # Select the state with the lowest heuristic value.

    def evaluate(self, states, parent_turn):
        """
        Evaluates a list of game states using the Minimax algorithm.

        This method evaluates a list of game states using the Minimax algorithm, which is a decision-making algorithm in
        game theory for minimizing the possible loss for a worst-case scenario. It assigns heuristic values to
        each state based on the algorithm's calculations.

        :param states: List of game states to evaluate.
        :param parent_turn: Indicates whose turn it is: True for the player trying to maximize and False
                            for the player trying to minimize.
        """
        for state in states:
            if state.can_claim_draw():
                state.h = 0.0  # Set the heuristic value to 0 if the game can be claimed as a draw.
            else:
                # Calculate heuristic value using Minimax.
                state.h = self.__minmax(state, self.max_depth - 1, not parent_turn)

    def __minmax(self, state, depth, turn):
        """
        Recursive helper method to perform the Minimax search.

        This private method performs a recursive Minimax search on a game tree to determine the heuristic
        value of a given game state.

        :param state: The current game state.
        :param depth: The current depth in the search.
        :param turn: Indicates whose turn it is: True for the player trying to maximize and False for the player
                    trying to minimize.
        :return: Heuristic value of the provided game state.
        """
        self.eval_count += 1  # Increment evaluation count.
        neighbors = self.game.neighbors(state)  # Get neighboring states from the current state.

        # Base cases: If the search depth is 0 or if the game is in an endgame state, return the heuristic value.
        if depth == 0 or state.is_endgame():
            return self.heuristic.h(state)

        if turn:
            value = -np.inf  # Initialize value for maximizing player to negative infinity.
            for child in neighbors:
                value = max(value, self.__minmax(child, depth - 1, False))  # Recursively maximize.
            return value
        else:
            value = np.inf  # Initialize value for minimizing player to positive infinity.
            for child in neighbors:
                value = min(value, self.__minmax(child, depth - 1, True))  # Recursively minimize.
            return value

    def search(self, state):
        """
        Initiates the Minimax search for a given game state.

        This method initializes the Minimax search process for a given game state.
        It calculates the heuristic values for the neighboring states and selects the best next state based
        on the Minimax algorithm.

        :param state: The game state to start the search from.
        :return:  Best next game state based on the Minimax algorithm.
        """
        neighbors = self.game.neighbors(state)  # Get neighboring states from the current state.
        self.evaluate(neighbors, state.turn())  # Calculate heuristic values for the neighbors.
        return self.pick(neighbors, state.turn())  # Select the best next state using the Minimax algorithm.


## MinMaxAlphaBetaPruning

In [10]:
class MinMaxAlphaBetaPruning:
    """
    Implementation of Minimax algorithm with Alpha-Beta pruning.

    Attributes:
        game: An instance of a game object that provides interface methods for the game state and its neighbors.
        heuristic: An instance of a heuristic object used to evaluate game states.
        max_depth: Maximum depth for the minimax search. Default is 1.
        prune_count: Count of the times pruning occurred during the search.
        eval_count: Count of the evaluations performed during the search.
    """

    def __init__(self, game, heuristic, max_depth=1):
        """
        Initializes an instance of the MinMaxAlphaBetaPruning class.
        :param game: The game for which the search is performed.
        :param heuristic: The heuristic to evaluate the game states.
        :param max_depth: Maximum depth of the search. Default is 1.
        """
        self.game = game
        self.heuristic = heuristic
        self.max_depth = max_depth
        self.prune_count = 0
        self.eval_count = 0

    @staticmethod
    def pick(states, parent_turn):
        """
        Picks the best state based on the heuristic values.

        This function evaluates a list of game states and selects the state that optimizes
        the current player's position.
        If it is the maximizing player's turn (parent_turn is True), the state with the highest heuristic
        value is chosen.
        Otherwise, if it is the minimizing player's turn (parent_turn is False), the state with the lowest heuristic
        value is chosen.

        :param states: List of game states to pick from.
        :param parent_turn: Indicates whose turn it is: True for maximizing player and False for minimizing player.
        :return: The best state based on the heuristic value.
        """
        if parent_turn:
            # If it's the maximizing player's turn, select the state with the highest heuristic value.
            return max(states, key=lambda state: state.h)
        else:
            # If it's the minimizing player's turn, select the state with the lowest heuristic value.
            return min(states, key=lambda state: state.h)

    def evaluate(self, states, parent_turn):
        """
        Evaluates a list of game states using the Minimax algorithm with Alpha-Beta pruning.

        This function evaluates a list of game states using the Minimax algorithm with Alpha-Beta pruning.
        It assigns a heuristic value to each state based on its evaluation at a specified depth in the game tree.
        The depth of the evaluation is determined by the 'max_depth' attribute of the object.

        :param states: List of game states to evaluate.
        :param parent_turn: Indicates whose turn it is: True for maximizing player and False for minimizing player.
        """
        for state in states:
            if state.can_claim_draw():
                # If the state can claim a draw, assign a heuristic value of 0.
                state.h = 0.0
            else:
                # Otherwise, use the Minimax algorithm with Alpha-Beta pruning to assign a heuristic value.
                state.h = self.__minmax_alpha_beta(state, self.max_depth - 1, -np.inf, np.inf, not parent_turn)

    def __minmax_alpha_beta(self, state, depth, alpha, beta, turn):
        """
        Recursive helper method to perform Minimax search with Alpha-Beta pruning.

        This private method performs a recursive Minimax search with Alpha-Beta pruning to find the optimal move
        in the game tree.
        It evaluates the provided game state and returns a heuristic value based on the current player's turn.

        :param state: Current game state.
        :param depth: Current depth in the search.
        :param alpha: Best already explored option for the maximizer.
        :param beta: Best already explored option for the minimizer.
        :param turn: Indicates whose turn it is: True for maximizing player and False for minimizing player.
        :return: Heuristic value of the provided game state.
        """
        self.eval_count += 1  # Count the number of state evaluations.
        neighbors = self.game.neighbors(state)  # Generate possible successor states.

        if depth == 0 or state.is_endgame():
            # Base case: If the maximum depth is reached or the state represents an endgame, return the heuristic value.
            return self.heuristic.h(state)

        if turn:  # Maximizing player
            value = -np.inf
            for neighbor in neighbors:
                value = max(value, self.__minmax_alpha_beta(neighbor, depth - 1, alpha, beta, False))
                alpha = max(alpha, value)  # Update alpha with the maximum value found so far.
                if alpha >= beta:  # Alpha-Beta pruning: Stop evaluating if alpha is greater than or equal to beta.
                    self.prune_count += 1  # Count pruned branches.
                    break
            return value
        else:  # Minimizing player
            value = np.inf
            for neighbor in neighbors:
                value = min(value, self.__minmax_alpha_beta(neighbor, depth - 1, alpha, beta, True))
                beta = min(beta, value)  # Update beta with the minimum value found so far.
                if beta <= alpha:  # Alpha-Beta pruning: Stop evaluating if beta is less than or equal to alpha.
                    self.prune_count += 1  # Count pruned branches.
                    break
            return value

    def search(self, state):
        """
        Initiates the Minimax search with Alpha-Beta pruning for a given game state.

        This method initializes the Minimax search with Alpha-Beta pruning to find the best next game state based on the
        current game state.
        It evaluates the neighboring states, chooses the best move, and returns the resulting game state.

        :param state: The game state to search from.
        :return: Best next game state based on the Minimax algorithm with Alpha-Beta pruning.
        """
        neighbors = self.game.neighbors(state)  # Generate possible successor states.
        self.evaluate(neighbors, state.turn())  # Evaluate the neighboring states using Minimax.
        return self.pick(neighbors, state.turn())  # Choose and return the best next game state.


# Method Main

In [11]:
import time


def main_chess_game():
    game = ChessGame()
    heuristic_a1 = HardBoardEvaluationChessGame()
    search_algorithm_a1 = MinMaxAlphaBetaPruning(game=game, heuristic=heuristic_a1, max_depth=2)
    search_algorithm_a2 = MinMaxAlphaBetaPruning(game=game, heuristic=heuristic_a1, max_depth=2)
    state = StateChessGame()
    agent1 = Agent(search_algorithm_a1, state)
    agent2 = Agent(search_algorithm_a2, state)
    turn_agent = 0
    move_agent_1 = 1
    move_agent_2 = 1
    start_time = time.time()
    print("The game of chess begins!")
    print(state.game_representation)
    while not state.is_endgame():
        if turn_agent % 2:
            state = agent2.do_action(state)
            move_agent_2 += 1
            print(state.game_representation)
            print("Agent 2 (BLACK) played the move:", state.move)
            print()
        else:
            state = agent1.do_action(state)
            move_agent_1 += 1
            print(state.game_representation)
            print("Agent 1 (WHITE) played the move:", state.move)
            print()
        turn_agent = turn_agent + 1

        if state is None:
            print("The agent was unable to resolve the issue")
            return

    end_time = time.time()
    print(f"Result in: {(end_time - start_time) * 1000:.2f}ms")
    print(
        f"OUTCOME: {state.game_representation.game_board.outcome().termination.name}")
    if state.game_representation.get_name_winner_player() is not None:
        print(f"Player Win: {state.game_representation.get_name_winner_player().upper()}")
    print(f"Number of Moves       (agent 1 WHITHE): {move_agent_1}")
    print(f"States evaluated      (agent 1 WHITHE): {agent1.search_algorithm.eval_count}")
    print(f"Pruning carried out   (agent 1 WHITHE): {agent1.search_algorithm.prune_count}")
    print()
    print(f"Number of Moves       (agent 2 BLACK): {move_agent_2}")
    print(f"States evaluated      (agent 2 BLACK): {agent2.search_algorithm.eval_count}")
    print(f"Pruning carried out   (agent 2 BLACK): {agent2.search_algorithm.prune_count}")


if __name__ == '__main__':
    main_chess_game()

The game of chess begins!
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . N . .
P P P P P P P P
R N B Q K B . R
Agent 1 (WHITE) played the move: g1f3
r n b q k b n r
p p p . p p p p
. . . . . . . .
. . . p . . . .
. . . . . . . .
. . . . . N . .
P P P P P P P P
R N B Q K B . R
Agent 2 (BLACK) played the move: d7d5
r n b q k b n r
p p p . p p p p
. . . . . . . .
. . . p . . . .
. . . . . . . P
. . . . . N . .
P P P P P P P .
R N B Q K B . R
Agent 1 (WHITE) played the move: h2h4
r n . q k b n r
p p p . p p p p
. . . . . . . .
. . . p . . . .
. . . . . . . P
. . . . . N . b
P P P P P P P .
R N B Q K B . R
Agent 2 (BLACK) played the move: c8h3
r n . q k b n r
p p p . p p p p
. . . . . . . .
. . . p . . . .
. . . . . . . P
. . . . . N . R
P P P P P P P .
R N B Q K B . .
Agent 1 (WHITE) played the move: h1h3
r n . q k b n r
